# Query examples to use on the knowledge graph

#### What are the potential targets for a given disease? 

In [ ]:
/* Direct relationships */
MATCH (disease:Disease {Name:"OVARIAN CANCER"})<-[diseaseToDrug:APPROVED_TO_TREAT]-(drug:Drug)
WITH disease, diseaseToDrug, drug as d
MATCH (drug:Drug)-[drugToGene]-(gene:Gene)
WHERE drug.Name=d.Name
RETURN DISTINCT disease, diseaseToDrug, d, drug, drugToGene, gene
;

/* Indirect relationships (one further hop) */
MATCH (disease:Disease {Name:"OVARIAN CANCER"})<-[diseaseToDrug:APPROVED_TO_TREAT]-(drug:Drug)
WITH disease, diseaseToDrug, drug as d
MATCH (drug:Drug)-[drugToGene]-(gene:Gene)
WHERE drug.Name=d.Name
WITH disease, diseaseToDrug, d, drug, drugToGene, gene as g
MATCH (gene:Gene)-[geneToGene]-(gene2:Gene)
WHERE gene.Symbol=g.Symbol
RETURN DISTINCT disease, diseaseToDrug, d, drug, drugToGene, g, gene, geneToGene, gene2 
;

#### How do two given genes potentially interact? 

In [ ]:
/* Direct relationships */
MATCH (g1:Gene {Symbol: "EGFR"})-[geneToGene]-(g2:Gene{Symbol: "TP53"})
RETURN DISTINCT type(geneToGene)
;

/* Indirect relationships (one further hop) */
MATCH (gene:Gene {Symbol: "EGFR"})-[geneToOther]-(othernode)-[otherToGene]-(gene2:Gene {Symbol: "TP53"})
RETURN gene, geneToOther, othernode, otherToGene, gene2 LIMIT 5
;

#### What drugs could be associated with a given gene? 

In [ ]:
MATCH (d:Drug)-[drugToGene]->(g:Gene{Symbol: "BCL2"})
RETURN DISTINCT d, type(drugToGene)
;

#### (1) What drugs have been approved to treat Acute Myeloid Leukemia?

#### (2) What are the targets for these drugs?

#### (3) What are the genes that interact with these target genes?

In [ ]:
/* (1) */
MATCH (drug:Drug)-[drugToDisease:APPROVED_TO_TREAT]->(disease:Disease {Name: "ACUTE MYELOGENOUS LEUKEMIA"})
RETURN DISTINCT drug, drugToDisease, disease

/* (2) */
MATCH (drug:Drug)-[drugToDisease:APPROVED_TO_TREAT]->(disease:Disease {Name: "ACUTE MYELOGENOUS LEUKEMIA"})
WITH disease. drugToDisease, drug as d
MATCH (drug:Drug)-[drugToGene:TARGETS]-(gene:Gene)
WHERE d.Name = drug.Name
RETURN DISTINCT disease, drugToDisease, d, drug, drugToGene, gene

/* (3) */
MATCH (drug:Drug)-[diseaseToDrug:APPROVED_TO_TREAT]->(disease:Disease {Name:"ACUTE MYELOGENOUS LEUKEMIA"})
WITH disease, diseaseToDrug, drug as d
MATCH (drug:Drug)-[drugToGene]-(gene:Gene)
WHERE d.Name = drug.Name 
WITH disease, diseaseToDrug, d, drug, drugToGene, gene as g
MATCH (gene:Gene)-[geneToGene]-(gene2:Gene)
WHERE g.Symbol = gene.Symbol
RETURN DISTINCT disease, diseaseToDrug, d, drug, drugToGene, g, gene, geneToGene, gene2 
;